In [25]:
import os
import numpy as np
import torch
import cv2
import torchvision.transforms as transforms
from torchvision.models import resnet50
from scipy.spatial.distance import cosine

In [26]:
#paths
ref_folder=r"D:\ConstructionSiteSafety\img"

In [ ]:
#load the pre-trained ResNet model
model=resnet50(pretrained=True)
model.eval()#set to evaluation mode

In [28]:
#Transform for preprocessing images
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)), #resize to ResNet input size
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224, 0.225])
])

In [29]:
#function to extfract features from an image
def extract_features(image):
    img_tensor=transform(image).unsqueeze(0)#apply transformatuions and add batch dimension
    with torch.no_grad():
        features=model(img_tensor).flatten().numpy()#extract features
        return features

In [30]:
#extract features from all reference helmet images
ref_features={}
for img_file in os.listdir(ref_folder):
    img_path=os.path.join(ref_folder,img_file)
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)#convert to rgb
    ref_features[img_file]=extract_features(img)

In [31]:
#Initialize the webcam
cap=cv2.VideoCapture(0)

In [32]:
#Load the pre-trained custom helmet detection model(you need to ensure this path is correct)
helmet_cascade_path=r"file:///D:/ConstructionSiteSafety/annotations.xml"
helmet_cascade=cv2.CascadeClassifier(helmet_cascade_path)

In [ ]:
#check if the cascade is loaded successfully
if helmet_cascade.empty():
    print("Error: Unable to load the helmet cascade file!")
else:
    print("Helmet cascade loaded successfully!")

while True:
    ret, frame=cap.read()
    if not ret:
        break

In [ ]:
w#convert the frame to grayscale for helmet detetction
gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

In [ ]:
#detect helmets in the frame using the custom cascade
helmets=helmet_cascade.detectMultiScale(gray_frame,scaleFactor=1.1,minNeighbors=5, minSize=(50,50))